<a href="https://colab.research.google.com/github/Frodo-Swaggins/Fake_Real_News_Classifier/blob/main/Fake_Real_News_Classification_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

SystemError: ignored

In [ ]:
#Read in the datasets
from google.colab import drive
import numpy as np
import pandas as pd

drive.mount('/content/drive')

copied_path_fake = '/content/drive/MyDrive/Fake.csv'
copied_path_real = '/content/drive/MyDrive/True.csv'

df_fake = pd.read_csv(copied_path_fake, encoding='latin-1')
df_real = pd.read_csv(copied_path_real, encoding='latin-1')

#drop all unnecassary info from the dataset
df_fake = df_fake.iloc[:,[1]]
df_real = df_real.iloc[:,[1]]

#Add in a column for real an fake labels
df_fake.insert(1,'Tag','fake')
df_real.insert(1,'Tag','real')

#Reduce the size of the dataset for training time purposes
df_fake=df_fake.drop(df_fake.index[8000:],axis=0)
df_real=df_real.drop(df_real.index[8000:],axis=0)

# Stack the DataFrames on top of each other
df = pd.concat([df_fake, df_real], axis=0)

print(df)
print(len(df))

Mounted at /content/drive
                                                   text   Tag
0     Donald Trump just couldn t wish all Americans ...  fake
1     House Intelligence Committee Chairman Devin Nu...  fake
2     On Friday, it was revealed that former Milwauk...  fake
3     On Christmas day, Donald Trump announced that ...  fake
4     Pope Francis used his annual Christmas Day mes...  fake
...                                                 ...   ...
7995  NEW YORK/LOS ANGELES (Reuters) - The first pre...  real
7996  WASHINGTON (Reuters) - Democratic presidential...  real
7997  (Reuters) - Republican presidential candidate ...  real
7998  WASHINGTON (Reuters) - The U.S. Senateâs sen...  real
7999  WASHINGTON (Reuters) - The FBI is investigatin...  real

[16000 rows x 2 columns]
16000


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('english')
from nltk.stem import WordNetLemmatizer
from textblob import Word
nltk.download('punkt')
nltk.download('wordnet')
import re                                  
import string                             
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Error loading english: Package 'english' not found in
[nltk_data]     index
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
def cleaning(df, stop_words):

    df['text'] = df['text'].apply(lambda x: ' '.join(x.lower() for x in x.split()))
    print(df)

    # Replacing the special characters

    df['text'] = df['text'].str.replace('[^\w\s]', '')
    print(df)

    # Replacing the digits/numbers

    df['text'] = df['text'].str.replace('\d', '')
    print(df)

    # Removing stop words

    df['text'] = df['text'].apply(lambda x: ' '.join(x for x in x.split() if x not in stop_words))
    print(df)

    # Lemmatization

    df['text'] = df['text'].apply(lambda x: ' '.join([Word(x).lemmatize() for x in x.split()]))
    print(df)

    return df

stop_words = stopwords.words('english')

In [ ]:
df = cleaning(df, stop_words)

                                                   text   Tag
0     donald trump wish american happy new year leav...  fake
1     house intelligence committee chairman devin nu...  fake
2     friday revealed former milwaukee sheriff david...  fake
3     christmas day donald trump announced would bac...  fake
4     pope francis used annual christmas day message...  fake
...                                                 ...   ...
7995  new yorklos angeles reuters first presidential...  real
7996  washington reuters democratic presidential can...  real
7997  reuters republican presidential candidate dona...  real
7998  washington reuters u senateâs senior democrati...  real
7999  washington reuters fbi investigating suspected...  real

[16000 rows x 2 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  


                                                   text   Tag
0     donald trump wish american happy new year leav...  fake
1     house intelligence committee chairman devin nu...  fake
2     friday revealed former milwaukee sheriff david...  fake
3     christmas day donald trump announced would bac...  fake
4     pope francis used annual christmas day message...  fake
...                                                 ...   ...
7995  new yorklos angeles reuters first presidential...  real
7996  washington reuters democratic presidential can...  real
7997  reuters republican presidential candidate dona...  real
7998  washington reuters u senateâs senior democrati...  real
7999  washington reuters fbi investigating suspected...  real

[16000 rows x 2 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]


                                                   text   Tag
0     donald trump wish american happy new year leav...  fake
1     house intelligence committee chairman devin nu...  fake
2     friday revealed former milwaukee sheriff david...  fake
3     christmas day donald trump announced would bac...  fake
4     pope francis used annual christmas day message...  fake
...                                                 ...   ...
7995  new yorklos angeles reuters first presidential...  real
7996  washington reuters democratic presidential can...  real
7997  reuters republican presidential candidate dona...  real
7998  washington reuters u senateâs senior democrati...  real
7999  washington reuters fbi investigating suspected...  real

[16000 rows x 2 columns]
                                                   text   Tag
0     donald trump wish american happy new year leav...  fake
1     house intelligence committee chairman devin nu...  fake
2     friday revealed former milwaukee sheri

In [ ]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

#Add one hot encoding for the vocabularly from the dataset
vocab_size = 10000
encoded_docs = [one_hot(d, vocab_size) for d in df['text']]
max_length = 120
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

#Add encoding for the labels from the dataset
lb=LabelEncoder()
encoded_label = lb.fit_transform(df['Tag'])

In [ ]:
from sklearn.model_selection import train_test_split 

y=pd.get_dummies(encoded_label)
X_train, X_test, y_train, y_test = train_test_split(padded_docs,y, test_size = 0.3, random_state = 42)

In [ ]:
embedding_dim=16
trunc_type= 'post'
oov_tok="<OOV>"

Simple_RNN_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                             input_length=max_length),
    tf.keras.layers.SimpleRNN(32),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(2, activation='sigmoid')
])
Simple_RNN_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           160000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                1568      
                                                                 
 dense (Dense)               (None, 10)                330       
                                                                 
 dense_1 (Dense)             (None, 2)                 22        
                                                                 
Total params: 161,920
Trainable params: 161,920
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Simple_RNN_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])

batch_size=2000
history = Simple_RNN_model.fit(X_train, y_train, epochs = 30, batch_size=batch_size, verbose = 'auto')

loss, accuracy, f1_score, precision, recall = Simple_RNN_model.evaluate(X_test, y_test, verbose=0)

Epoch 1/30
6/6 [==============================] - 3s 226ms/step - loss: 0.6702 - acc: 0.5038 - f1_m: 0.4491 - precision_m: 0.5939 - recall_m: 0.3635
Epoch 2/30
6/6 [==============================] - 1s 210ms/step - loss: 0.5943 - acc: 0.5507 - f1_m: 0.5757 - precision_m: 0.7304 - recall_m: 0.4769
Epoch 3/30
6/6 [==============================] - 1s 215ms/step - loss: 0.5224 - acc: 0.7479 - f1_m: 0.7016 - precision_m: 0.8940 - recall_m: 0.5782
Epoch 4/30
6/6 [==============================] - 1s 211ms/step - loss: 0.4716 - acc: 0.9323 - f1_m: 0.7907 - precision_m: 0.9505 - recall_m: 0.6773
Epoch 5/30
6/6 [==============================] - 1s 214ms/step - loss: 0.4437 - acc: 0.9511 - f1_m: 0.6579 - precision_m: 0.9559 - recall_m: 0.5019
Epoch 6/30
6/6 [==============================] - 1s 211ms/step - loss: 0.4321 - acc: 0.9490 - f1_m: 0.6418 - precision_m: 0.9310 - recall_m: 0.4898
Epoch 7/30
6/6 [==============================] - 1s 206ms/step - loss: 0.4396 - acc: 0.9180 - f1_m: 0.594

In [ ]:
print('loss : ',loss)
print('accuracy : ',accuracy)
print('f1 score : ',f1_score)
print('precision : ',precision)
print('recall : ',recall)

loss :  0.12412045896053314
accuracy :  0.9839583039283752
f1 score :  0.9837433099746704
precision :  0.9839516282081604
recall :  0.9835416674613953


In [ ]:
GRU_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                             input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(2, activation='sigmoid')
])
GRU_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 120, 16)           160000    
                                                                 
 bidirectional (Bidirectiona  (None, 64)               9600      
 l)                                                              
                                                                 
 dense_2 (Dense)             (None, 10)                650       
                                                                 
 dense_3 (Dense)             (None, 2)                 22        
                                                                 
Total params: 170,272
Trainable params: 170,272
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

GRU_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])

batch_size=2000
history = GRU_model.fit(X_train, y_train, epochs = 30, batch_size=batch_size, verbose = 'auto')

loss, accuracy, f1_score, precision, recall = GRU_model.evaluate(X_test, y_test, verbose=0)

Epoch 1/30
6/6 [==============================] - 11s 1s/step - loss: 0.6894 - acc: 0.5049 - f1_m: 0.4388 - precision_m: 0.5003 - recall_m: 0.4064
Epoch 2/30
6/6 [==============================] - 6s 1s/step - loss: 0.6814 - acc: 0.5126 - f1_m: 0.5355 - precision_m: 0.5238 - recall_m: 0.5483
Epoch 3/30
6/6 [==============================] - 8s 1s/step - loss: 0.6700 - acc: 0.6588 - f1_m: 0.7156 - precision_m: 0.6255 - recall_m: 0.8369
Epoch 4/30
6/6 [==============================] - 7s 1s/step - loss: 0.6521 - acc: 0.7656 - f1_m: 0.7601 - precision_m: 0.6483 - recall_m: 0.9185
Epoch 5/30
6/6 [==============================] - 7s 1s/step - loss: 0.6239 - acc: 0.7985 - f1_m: 0.7620 - precision_m: 0.6490 - recall_m: 0.9226
Epoch 6/30
6/6 [==============================] - 7s 1s/step - loss: 0.5807 - acc: 0.8587 - f1_m: 0.7666 - precision_m: 0.6511 - recall_m: 0.9317
Epoch 7/30
6/6 [==============================] - 6s 1s/step - loss: 0.5168 - acc: 0.8756 - f1_m: 0.7656 - precision_m: 0.6

In [ ]:
print('loss : ',loss)
print('accuracy : ',accuracy)
print('f1 score : ',f1_score)
print('precision : ',precision)
print('recall : ',recall)

loss :  0.023762129247188568
accuracy :  0.9941666722297668
f1 score :  0.9940705299377441
precision :  0.9937752485275269
recall :  0.9943749904632568


In [ ]:
LSTM_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(2, activation='sigmoid')
])
LSTM_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 120, 16)           160000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               12544     
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 10)                650       
                                                                 
 dense_5 (Dense)             (None, 2)                 22        
                                                                 
Total params: 173,216
Trainable params: 173,216
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

LSTM_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])

batch_size=2000
history = LSTM_model.fit(X_train, y_train, epochs = 30, batch_size=batch_size, verbose = 'auto')

loss, accuracy, f1_score, precision, recall = LSTM_model.evaluate(X_test, y_test, verbose=0)

Epoch 1/30
6/6 [==============================] - 11s 1s/step - loss: 0.6914 - acc: 0.5009 - f1_m: 0.5024 - precision_m: 0.5159 - recall_m: 0.4903
Epoch 2/30
6/6 [==============================] - 7s 1s/step - loss: 0.6853 - acc: 0.5063 - f1_m: 0.5187 - precision_m: 0.5388 - recall_m: 0.5005
Epoch 3/30
6/6 [==============================] - 7s 1s/step - loss: 0.6745 - acc: 0.5487 - f1_m: 0.5677 - precision_m: 0.6570 - recall_m: 0.5001
Epoch 4/30
6/6 [==============================] - 7s 1s/step - loss: 0.6511 - acc: 0.5724 - f1_m: 0.5674 - precision_m: 0.6513 - recall_m: 0.5027
Epoch 5/30
6/6 [==============================] - 7s 1s/step - loss: 0.5919 - acc: 0.6383 - f1_m: 0.5698 - precision_m: 0.6600 - recall_m: 0.5014
Epoch 6/30
6/6 [==============================] - 7s 1s/step - loss: 0.4491 - acc: 0.8231 - f1_m: 0.6808 - precision_m: 0.8017 - recall_m: 0.5921
Epoch 7/30
6/6 [==============================] - 7s 1s/step - loss: 0.3883 - acc: 0.9616 - f1_m: 0.9383 - precision_m: 0.9

In [ ]:
print('loss : ',loss)
print('accuracy : ',accuracy)
print('f1 score : ',f1_score)
print('precision : ',precision)
print('recall : ',recall)

loss :  0.05922591686248779
accuracy :  0.9706249833106995
f1 score :  0.9706249833106995
precision :  0.9706249833106995
recall :  0.9706249833106995
